In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:15pt;}
div.text_cell_render.rendered_html{font-size:18pt;}
div.text_cell_render ul li{font-size:22pt; line-height:30px;}
div.output {font-size:22pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:22pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:22pt;padding:5px;}
table.dataframe{font-size:22px;}
</style>
"""))

In [2]:
# ===============================
# 📌 기본 라이브러리
# ===============================
import pandas as pd
import numpy as np

# ===============================
# 📌 시각화 라이브러리
# ===============================
import matplotlib.pyplot as plt
import seaborn as sns

# ===============================
# 📌 경고 메시지 제거 (대용량 CSV 처리 시 깔끔)
# ===============================
import warnings
warnings.filterwarnings("ignore")

# ===============================
# 📌 한글 폰트 설정 (Windows)
# ===============================
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

In [3]:
# ===============================
# 📌 CSV에서 불러올 컬럼 지정
# (불필요한 컬럼 제거 → 메모리 절약)
# ===============================
use_cols = [
    '인허가일자',
    '상세영업상태명',
    '폐업일자',
    '소재지전체주소',
    '도로명전체주소',
    '업태구분명',
    '좌표정보x(epsg5174)',
    '좌표정보y(epsg5174)',
    '사업장명'
]


In [5]:
# ===============================
# 📌 데이터 로드
# ===============================
df = pd.read_csv(
    r"C:\ai\lecNote\1st_Project\data\6110000_서울특별시_07_24_04_P_일반음식점.csv",
    encoding="cp949",
    usecols=use_cols,
    low_memory=False
)

In [6]:
# 소재지 주소가 있으면 우선 사용, 없으면 도로명 주소 사용
df['주소'] = df['소재지전체주소'].fillna(df['도로명전체주소'])

In [7]:
df['주소'].isna().sum()

0

In [8]:
# 주소 문자열에서 '○○구' 패턴 추출
df['구'] = df['주소'].str.extract(r'(서울특별시\s*)?(\w+구)')[1]
# 구 정보 없는 행 제거
df[['주소', '구']].head()

,주소,구
0,서울특별시 강북구 수유동 229-46,강북구
1,서울특별시 종로구 관훈동 155-2,종로구
2,서울특별시 종로구 관훈동 155-2,종로구
3,서울특별시 종로구 인사동 241,종로구
4,서울특별시 양천구 신월동 60-33,양천구


In [9]:
# '구' 결측치 제거
df['구'].isna().sum()
df = df[df['구'].notna()].copy()

In [20]:
# 서울특별시 데이터만 유지 (타 지역 자동 제거)
df = df[df['주소'].str.contains('서울', na=False)].copy()

In [21]:
# 문자열 → datetime 변환
df['인허가일자'] = pd.to_datetime(df['인허가일자'], errors='coerce')
df['폐업일자'] = pd.to_datetime(df['폐업일자'], errors='coerce')

# 인허가일자 없는 행 제거
df = df[df['인허가일자'].notna()].copy()

# ✅ 기간 필터: 2000-02-01 ~ 2022-11-30 (둘 다 포함)
start = pd.Timestamp("2000-02-01")
end   = pd.Timestamp("2022-11-30")
df = df[df["인허가일자"].between(start, end, inclusive="both")].copy()

# 좌표 결측치 제거
df = df[
    df['좌표정보x(epsg5174)'].notna() &
    df['좌표정보y(epsg5174)'].notna()
].copy()

In [22]:
# 창업월, 창업연도
df['창업월'] = df['인허가일자'].dt.month
df['창업연도'] = df['인허가일자'].dt.year

In [23]:
# 영업 기간 계산 
df['영업일수'] = (
    df['폐업일자'].fillna(pd.Timestamp.today())
    - df['인허가일자']
).dt.days

In [24]:
#3년 이내 폐업 여부
df['폐업_3년이내'] = (
    (df['폐업일자'].notna()) &
    (df['영업일수'] <= 365 * 3)
).astype(int)

In [25]:
#문자열의 앞뒤 공백을 제거
df['업태'] = df['업태구분명'].str.strip()

In [26]:
# 업태 빈도
type_counts = df['업태'].value_counts()

# 10개 이하 업태 목록
rare_types = type_counts[type_counts <= 10].index

rare_types

Index(['커피숍', '기타 휴게음식점', '일반조리판매', '다방', '제과점영업', '식품소분업'], dtype='object')

In [30]:
# 업태 빈도
type_counts = df['업태'].value_counts()

# 10개 이하 업태 목록
rare_types = type_counts[type_counts <= 10].index

rare_types

Index(['커피숍', '기타 휴게음식점', '일반조리판매', '다방', '제과점영업', '식품소분업'], dtype='object')

In [31]:
# 표본 10개 이하 업태 → 기타로 통합
df['업태_정리'] = df['업태'].replace(rare_types, '기타')

In [32]:
KEYWORD_GROUPS = {
    '카페': ['카페', '커피', 'coffee', 'COFFEE','케이크', '까페', 'cafe', 'CAFÉ', '다방', '샐러드', '프레소', '티', '차', '디저트'],
    '한식': ['식당', '찜', '전골', '전', '음식점', '밥상', '한식', '순대', '찌개', '구이' ,'곱창', '국수', '고기', '구이', '국', '밥', '마당', '옥', '관', '떡볶이', '김밥'],
    '주점/치킨': ['크리스피', 'KFC', 'kfc', '치킨', '비비큐', 'bbq',  'BBQ', '후라이드', '술집', '주점', '포장마차', '술', '호프', '건배', '맥주', '포차','펍','pup' ,'PUP', '주점', '바', '룸' ,'와인', '포차','소주', '맥주', '아지트', '잔', '위스키', '클럽'],
    '양식/외식': ['베이커리', '빵', '아웃백'],
    '중/일식': ['마라탕','양꼬치','샤브']
}

In [33]:
def reclassify_etc(row):
    if row['업태_정리'] != '기타':
        return row['업태_정리']

    text = f"{row['사업장명']} {row['업태']}".lower()

    for group, keywords in KEYWORD_GROUPS.items():
        for kw in keywords:
            if kw.lower() in text:
                return group

    return '기타'

df['업태_최종'] = df.apply(reclassify_etc, axis=1)

df['업태_최종'].value_counts()

한식                 117973
호프/통닭               28185
분식                  27681
기타                  22912
경양식                 21630
일식                  12785
중국식                  8145
통닭(치킨)               7713
까페                   7068
카페                   6618
정종/대포집/소주방           5853
패스트푸드                2897
주점/치킨                2335
외국음식전문점(인도,태국등)      2226
식육(숯불구이)             1448
김밥(도시락)              1427
횟집                   1262
뷔페식                  1120
감성주점                  351
탕류(보신용)               298
라이브카페                 288
패밀리레스트랑               258
출장조리                  240
전통찻집                  213
양식/외식                 165
냉면집                   153
중/일식                  143
키즈카페                  102
복어취급                   65
이동조리                   17
Name: 업태_최종, dtype: int64